### **Tahap 4 Disusun Oleh** 👨‍🎓✒️
>**Nama: Fitra Romeo Winky**

>**NIM: 202210370311056**

>**Nama: Muhammad Aunul Hakim**

>**NIM: 202210370311073**

>**Kelas: C**

>**Mata Kuliah: Penalaran Komputer**

>**Jurusan: Informatika**

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
import numpy as np
import os
import json
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
import torch

# Path
BASE_DIR = '/content/drive/MyDrive/SEMESTER 6/Penalaran Komputer/UAS_Penalaran Komputer'
DATA_PATH = os.path.join(BASE_DIR, 'data/processed/cases.csv')
RESULT_PATH = os.path.join(BASE_DIR, 'data/results/predictions.csv')

# Load data
df = pd.read_csv(DATA_PATH)
print(f"Data loaded: {len(df)} kasus")

# Buat retrieval_text jika belum ada
if 'retrieval_text' not in df.columns:
    text_columns = ['ringkasan_fakta', 'argumen_hukum', 'pihak']

    # Gabungkan kolom yang tersedia
    combined_parts = []
    for col in text_columns:
        if col in df.columns:
            combined_parts.append(df[col].fillna('').astype(str))

    if combined_parts:
        df['combined_text'] = combined_parts[0]
        for part in combined_parts[1:]:
            df['combined_text'] = df['combined_text'] + ' ' + part
    else:
        df['combined_text'] = ''

    # Gunakan text_full jika ada
    if 'text_full' in df.columns:
        df['retrieval_text'] = df['text_full'].fillna(df['combined_text'])
    else:
        df['retrieval_text'] = df['combined_text']



Data loaded: 31 kasus


In [38]:
# Bangun dictionary {case_id: solusi}
case_solutions = {}
for _, row in df.iterrows():
    case_id = row['case_id']
    if pd.notna(row.get('amar_putusan')) and str(row['amar_putusan']).strip():
        solusi = str(row['amar_putusan'])
    else:
        solusi = str(row.get('ringkasan_fakta', 'Tidak tersedia'))
    case_solutions[case_id] = solusi.strip()[:500]  # potong maksimal 500 karakter


In [39]:
# Representasi TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words=None)
tfidf_matrix = vectorizer.fit_transform(df['retrieval_text'].fillna(''))


In [40]:
def retrieve_similar_cases(query_text, top_k=5):
    """Lakukan pencarian kasus serupa dengan TF-IDF"""
    query_vector = vectorizer.transform([query_text])
    sims = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = sims.argsort()[-top_k:][::-1]
    return [(df.iloc[i]['case_id'], sims[i]) for i in top_indices]

def predict_outcome(query_text, method='majority', top_k=5):
    """Prediksi solusi dari top-k hasil retrieval"""
    top_k_cases = retrieve_similar_cases(query_text, top_k)
    solusi_list = [case_solutions[case_id] for case_id, _ in top_k_cases]

    if method == 'majority':
        return Counter(solusi_list).most_common(1)[0][0], [cid for cid, _ in top_k_cases]

    elif method == 'weighted':
        weighted_counter = {}
        for (case_id, score) in top_k_cases:
            solusi = case_solutions[case_id]
            weighted_counter[solusi] = weighted_counter.get(solusi, 0) + score
        return max(weighted_counter.items(), key=lambda x: x[1])[0], [cid for cid, _ in top_k_cases]

    else:
        raise ValueError("Method harus 'majority' atau 'weighted'")


In [42]:
queries = [
    {"query_id": "query_001", "text": "Tindak penganiayaan yang menyebabkan luka ringan"},
    {"query_id": "query_002", "text": "Kasus penganiayaan dalam rumah tangga"},
    {"query_id": "query_003", "text": "Kekerasan fisik akibat perselisihan keluarga"},
    {"query_id": "query_004", "text": "Penganiayaan saat bentrok antar warga"},
    {"query_id": "query_005", "text": "Penyerangan yang mengakibatkan korban luka berat"},
]

results.append({
    "query_id": q['query_id'],
    "query_text": q['text'],
    "predicted_solution": pred_sol,
    "top_5_case_ids": ";".join(str(case_id) for case_id in top_cases)
})



In [43]:
df_results = pd.DataFrame(results)
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)
df_results.to_csv(RESULT_PATH, index=False, encoding='utf-8')
print(f"✔️ Hasil prediksi disimpan ke: {RESULT_PATH}")


✔️ Hasil prediksi disimpan ke: /content/drive/MyDrive/SEMESTER 6/Penalaran Komputer/UAS_Penalaran Komputer/data/results/predictions.csv
